### **Prerequisites:**

 - access for given profile to *SageMaker* should be configured with *AmazonS3FullAccess*, *AmazonSageMakerFullAccess* and *AmazonEC2ContainerRegistryFullAccess* policies (done via **aws configure** in the profile in which jupyter is launched)
 - a special role with *AmazonSageMakerFullAccess* policy
 - a special bucket in S3 created a-priori
 - this role and the name of this bucket should be placed in the file **config.yml** put near this notebook with the following content:
 
 ```
    default-bucket: .........
    sagemaker-role: .........
 ```
 

In [ ]:
import tarfile
import yaml
import sagemaker
from sagemaker.pytorch import PyTorchModel
from sagemaker.predictor import json_serializer, json_deserializer
from sagemaker import get_execution_role


with open('./config.yml', 'r') as stream:
    config_dict = yaml.safe_load(stream)
sagemaker_session = sagemaker.Session(default_bucket=config_dict['default-bucket'])
sagemaker_role = config_dict['sagemaker-role']
# Uncomment the following line only in the case this notebook is launched from SageMaker notebook instance
# sagemaker_role = get_execution_role()

bucket = sagemaker_session.default_bucket()

### Deploying NST model

In [ ]:
if len(sagemaker_session.list_s3_files(bucket=bucket, key_prefix='sagemaker/NST/models/vgg19_state_dict.tar.gz')) > 0:
    vgg19_state_dict_model_data = f's3://{bucket}/sagemaker/NST/models/vgg19_state_dict.tar.gz'
else:
    import torch
    from torchvision import models

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    cnn = models.vgg19(pretrained=True).features.to(device).eval()
    torch.save(cnn.state_dict(), 'vgg19_state_dict.model')
    with tarfile.open('vgg19_state_dict.tar.gz', 'w:gz') as f:
        f.add('vgg19_state_dict.model')
    vgg19_state_dict_model_data = sagemaker_session.upload_data(path='./vgg19_state_dict.tar.gz', key_prefix='sagemaker/NST/models')

In [ ]:
nst_model = PyTorchModel(model_data=vgg19_state_dict_model_data,
                     role=sagemaker_role,
                     framework_version='1.5.0',
                     py_version='py3',
                     source_dir='./sagemaker/nst/',
                     entry_point='nst.py',
                     name='NeuralStyleTransfer',
                     env = {'SAGEMAKER_MODEL_SERVER_TIMEOUT' : '1200'}
                    )

In [ ]:
nst_predictor = nst_model.deploy(initial_instance_count = 1,
                           instance_type = 'ml.p2.xlarge', #ml.c4.xlarge
                           serializer=json_serializer,
                           deserializer=json_deserializer,
                           endpoint_name='NeuralStyleTransfer'
                          )

-------------------!

In [ ]:
# Uncomment and call what deployed predictor is not necessary
# nst_predictor.delete_endpoint()

### Deploying pretrained CycleGAN models (https://github.com/mashyko/pytorch-CycleGAN-and-pix2pix)

In [ ]:
PRETRAINED_MODELS_URL = 'http://efrosgans.eecs.berkeley.edu/cyclegan/pretrained_models/'

STYLE_NAMES_LIST = ['style_monet', 'style_vangogh', 'style_cezanne', 'style_ukiyoe']

In [ ]:
if len(sagemaker_session.list_s3_files(bucket=bucket, key_prefix='sagemaker/NST/models/cyclegan.tar.gz')) > 0:
    cyclegan_model_data = f's3://{bucket}/sagemaker/NST/models/cyclegan.tar.gz'
else:
    import pathlib
    import io
    import requests
    import urllib
    
    with tarfile.open('cyclegan.tar.gz', 'w:gz') as f:
        for style_name in STYLE_NAMES_LIST:
            model_file_name = f'{style_name}.pth'
            model_file_path = pathlib.Path(model_file_name)
            if not model_file_path.exists():
                urllib.request.urlretrieve(urllib.parse.urljoin(PRETRAINED_MODELS_URL, model_file_name), model_file_path)
            f.add(model_file_name)
    cyclegan_model_data = sagemaker_session.upload_data(path='./cyclegan.tar.gz', key_prefix='sagemaker/NST/models')

In [ ]:
cyclegan_model = PyTorchModel(model_data=cyclegan_model_data,
                     role=sagemaker_role,
                     framework_version='1.5.0',
                     py_version='py3',
                     source_dir='./sagemaker/cyclegan/',
                     entry_point='cyclegan.py',
                     name='CycleGANStyleTransfer',
                    )

In [ ]:
cyclegan_predictor = cyclegan_model.deploy(initial_instance_count = 1,
                           instance_type = 'ml.m5.large',
                           serializer=json_serializer,
                           deserializer=json_deserializer,
                           endpoint_name='CycleGANStyleTransfer'
                          )

-------------!

In [ ]:
# Uncomment and call what deployed predictor is not necessary
# cyclegan_predictor.delete_endpoint()